In [1]:
import numpy as np
from numpy.linalg import inv
from astropy.io import fits
import os
import re
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [2]:
# Theta from 0 to 180 degrees in increments of 5 degrees (for the first QWP). The second QWP rotates at 5 times this rate (25 degree increments)
theta = np.linspace(0, np.pi, 37)
longtheta = np.linspace(0, np.pi, 46)
degtheta = theta*180/np.pi

In [3]:
# Mueller matrix for a linear polarizer, with angle a between transmission axis and horizontal (radians)
def linear_polarizer(a):
    M01 = np.cos(2*a)
    M02 = np.sin(2*a)
    M10 = np.cos(2*a)
    M11 = np.cos(2*a)**2
    M12 = np.cos(2*a)*np.sin(2*a)
    M20 = np.sin(2*a)
    M21 = np.cos(2*a)*np.sin(2*a)
    M22 = np.sin(2*a)**2

    return 0.5*np.array([[1, M01, M02, 0], 
                         [M10, M11, M12, 0], 
                         [M20, M21, M22, 0], 
                         [0, 0, 0, 0]])

In [4]:
# Mueller matrix for a linear retarder (waveplate). Angle of fast axis a, retardance r in radians
def linear_retarder(a, r):
    M11 = np.cos(2*a)**2 + np.cos(r)*np.sin(2*a)**2
    M12 = np.cos(2*a)*np.sin(2*a)*(1-np.cos(r))
    M13 = -np.sin(2*a)*np.sin(r)
    M21 = M12
    M22 = np.sin(2*a)**2 + np.cos(2*a)**2*np.cos(r)
    M23 = np.cos(2*a)*np.sin(r)
    M31 = -M13
    M32 = -M23
    M33 = np.cos(r)

    return np.array([[1, 0, 0, 0], 
                     [0, M11, M12, M13], 
                     [0, M21, M22, M23], 
                     [0, M31, M32, M33]])

In [5]:
# Sorting function for extracting filenames based on last number in the filename (the angle of rotation)
def extract_number(filename):
    match = re.findall(r'\d+(?:\.\d+)?', filename)
    if match:
        return float(match[-1])

In [6]:
# Function to subtract dark frames from raw frames. The new reduced images are saved to a different folder
def dark_subtraction(image_file, dark_file, old_directory, new_directory):
    # Open the dark image and extract pixel values
    fits.open(dark_file)
    dark = fits.getdata(dark_file)
    dark_median = np.median(dark, axis=0)

    # Search through the desired raw data folder
    for filename in os.listdir(old_directory):
        if filename.startswith(image_file):                                # Call specific files starting with the desired name
            with fits.open(os.path.join(old_directory, filename)) as hdul:
                img_data = hdul[0].data
                img_median = np.median(img_data, axis=0)
                reduced_data = img_median - dark_median

            # Save the newly reduced image to a reduced data folder
            new_filename = f"Reduced_{filename}"
            new_filepath = os.path.join(new_directory, new_filename)
            fits.writeto(new_filepath, reduced_data, overwrite=True)

In [7]:
# Get intensity values from each spot in the reduced images. reduced_filename should just be the start of the name (leave out the last number, the angle). 
def extract_intensities(reduced_filename, reduced_folder, lcenter, rcenter, maxradius, cutoff=5000):
    I_left = np.array([])
    I_right = np.array([])
    bad_indices = np.array([])

    for filename in sorted(os.listdir(reduced_folder), key = extract_number):
        if filename.startswith(reduced_filename):
            with fits.open(os.path.join(reduced_folder, filename)) as hdul:
                reduced_img_data = hdul[0].data
                ys, xs, = np.indices(reduced_img_data.shape)
                lradius = np.sqrt((ys-lcenter[0])**2+(xs-lcenter[1])**2)
                rradius = np.sqrt((ys-rcenter[0])**2+(xs-rcenter[1])**2)

                lbackground_mask = (lradius > 20) & (lradius < 26)
                rbackground_mask = (rradius > 20) & (rradius < 26)   # Index the background around each spot, take the median value

                background_lmedian = np.median(reduced_img_data[lbackground_mask])
                background_rmedian = np.median(reduced_img_data[rbackground_mask])

                lflux = np.sum(reduced_img_data[lradius < maxradius] - background_lmedian)   # Now take the flux with the background mask subtracted
                rflux = np.sum(reduced_img_data[rradius < maxradius] - background_rmedian)
                I_left = np.append(I_left, lflux)
                I_right = np.append(I_right, rflux)

                if lflux+rflux < cutoff:
                    print("Warning: low flux detected, check the image " + filename + ", index: " + str(sorted(os.listdir(reduced_folder), key = extract_number).index(filename)))
                    bad_indices = np.append(bad_indices, sorted(os.listdir(reduced_folder), key = extract_number).index(filename))
                else:
                    continue 

    return I_left, I_right, bad_indices

In [8]:
# Gives the condition number of eventual Mueller matrix (made by Jaren)
def condition_number(matrix):
    minv = np.linalg.pinv(matrix)

    # Compute maximum norm
    norm = np.linalg.norm(matrix, ord=np.inf)
    ninv = np.linalg.norm(minv, ord=np.inf)

    return norm*ninv

In [9]:
# Function to compute the Mueller matrix of a sample based on DRRP intensity measurements. Big thanks to Jaren for this part!
def original_full_mueller_polarimetry(thetas, I_meas=1, LPA_angle=0, return_condition_number=False, M_in=None):
    nmeas = len(thetas)
    Wmat = np.zeros([nmeas, 16])
    Pmat = np.zeros([nmeas])
    th = thetas

    for i in range(nmeas):
        # Mueller Matrix of generator (linear polarizer and a quarter wave plate)
        Mg = linear_retarder(th[i], np.pi/2) @ linear_polarizer(0)

        # Mueller Matrix of analyzer (one channel of the Wollaston prism is treated as a linear polarizer. The right spot is horizontal (0) and the left spot is vertical(pi/2))
        Ma = linear_polarizer(LPA_angle) @ linear_retarder(th[i]*5, np.pi/2)

        # Data reduction matrix. Taking the 0 index ensures that intensity is the output
        Wmat[i,:] = np.kron(Ma[0,:], Mg[:,0])

        # M_in is some example Mueller matrix. Providing this input will test theoretical Mueller matrix. Otherwise, the raw data is used
        if M_in is not None:
            Pmat[i] = (Ma[0,:] @ M_in @ Mg[:,0]) * I_meas
        else:
            Pmat[i] = I_meas[i]

    # Compute Mueller matrix using Moore-Penrose pseudo invervse
    M = np.linalg.pinv(Wmat) @ Pmat
    M = np.reshape(M,[4,4])

    if return_condition_number == True:
        return M, condition_number(Wmat)
    else:
        return M

In [10]:
# Function to compute the Mueller matrix of a sample based on DRRP intensity measurements and calibration parameters
def calibrated_full_mueller_polarimetry(thetas, a1, a2, w1, w2, r1, r2, I_meas=1, LPA_angle=0, return_condition_number=False, M_in=None):
    nmeas = len(thetas)
    Wmat = np.zeros([nmeas, 16])
    Pmat = np.zeros([nmeas])
    th = thetas

    for i in range(nmeas):
        # Mueller Matrix of generator (linear polarizer and a quarter wave plate)
        Mg = linear_retarder(th[i]+w1, np.pi/2+r1) @ linear_polarizer(0+a1)

        # Mueller Matrix of analyzer (one channel of the Wollaston prism is treated as a linear polarizer. The right spot is horizontal (0) and the left spot is vertical(pi/2))
        Ma = linear_polarizer(LPA_angle+a2) @ linear_retarder(th[i]*5+w2, np.pi/2+r2)

        # Data reduction matrix. Taking the 0 index ensures that intensity is the output
        Wmat[i,:] = np.kron(Ma[0,:], Mg[:,0])

        # M_in is some example Mueller matrix. Providing this input will test theoretical Mueller matrix. Otherwise, the raw data is used
        if M_in is not None:
            Pmat[i] = (Ma[0,:] @ M_in @ Mg[:,0]) * I_meas
        else:
            Pmat[i] = I_meas[i]

    # Compute Mueller matrix using Moore-Penrose pseudo invervse
    M = np.linalg.pinv(Wmat) @ Pmat
    M = np.reshape(M,[4,4])

    if return_condition_number == True:
        return M, condition_number(Wmat)
    else:
        return M

In [11]:
# Define the identity matrix and other matrices which are useful for the Mueller calculus
M_identity = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
A = np.array([1, 0, 0, 0])
B = np.array([[1], [0], [0], [0]])

In [12]:
# This is the full Mueller matrix equation for our setup. The output is a list, useful for curve fitting. Variables with 1 refer to the generator, 2 refers to analyzer. 
def calibration_function(t, a1, a2, w1, w2, r1, r2):
    prediction = [None]*len(t)
    for i in range(len(t)):
        prediction[i] = float(A @ linear_polarizer(a2) @ linear_retarder(5*t[i]+w2, np.pi/2+r2) @ M_identity @ linear_retarder(t[i]+w1, np.pi/2+r1) @ linear_polarizer(a1) @ B)
    return prediction

In [13]:
# Calibration function designed for data from the left spot, which is the vertial alignment. This changes the angle of the analyzing LP
def vertical_calibration_function(t, a1, a2, w1, w2, r1, r2):
    prediction = [None]*len(t)
    for i in range(len(t)):
        prediction[i] = float(A @ linear_polarizer(a2+np.pi/2) @ linear_retarder(5*t[i]+w2, np.pi/2+r2) @ M_identity @ linear_retarder(t[i]+w1, np.pi/2+r1) @ linear_polarizer(a1) @ B)
    return prediction

In [14]:
# Basically the same as above, but with an optional input matrix to simulate data
def output_simulation_function(t, a1, a2, w1, w2, r1, r2, LPA_angle=0, M_in=None):
    if M_in is None:
        M = M_identity
    else:
        M = M_in

    prediction = [None]*len(t)
    for i in range(len(t)):
        prediction[i] = float(A @ linear_polarizer(LPA_angle+a2) @ linear_retarder(5*t[i]+w2, np.pi/2+r2) @ M @ linear_retarder(t[i]+w1, np.pi/2+r1) @ linear_polarizer(a1) @ B)
    return prediction

In [15]:
# After testing, each of the above functions works individually. Now combine them into one function to rule them all
# Finds the mueller matrix derived from each channel separately, then averages the two retardances found this way
# First three inputs must come from the calibration data, last three inputs correspond to the HWP sample
def ultimate_polarimetry(cal_angles, cal_left_intensity, cal_right_intensity, sample_angles, sample_left_intensity, sample_right_intensity):
    initial_guess = [0, 0, 0, 0, 0, 0]
    parameter_bounds = ([-np.pi, -np.pi, -np.pi, -np.pi, -np.pi/2, -np.pi/2], [np.pi, np.pi, np.pi, np.pi, np.pi/2, np.pi/2])

    # Find parameters from calibration of the left spot
    lnormalized_intensity = cal_left_intensity/(2*max(cal_left_intensity))
    lpopt, lpcov = curve_fit(vertical_calibration_function, cal_angles, lnormalized_intensity, p0=initial_guess, bounds=parameter_bounds)
    print(lpopt, "Left parameters for a1, a2, w1, w2, r1, and r2. 1 for generator, 2 for analyzer")
    #print(np.sqrt(np.diag(lpcov)))

    # Find parameters from calibration of the right spot
    rnormalized_intensity = cal_right_intensity/(2*max(cal_right_intensity))
    rpopt, rpcov = curve_fit(calibration_function, cal_angles, rnormalized_intensity, p0=initial_guess, bounds=parameter_bounds)
    print(rpopt, "Right parameters for a1, a2, w1, w2, r1, and r2. 1 for generator, 2 for analyzer")
    #print(np.sqrt(np.diag(rpcov)))

    # Optional print the calibration matrices (should be close to identity) to see how well the parameters compensate
    MlCal = calibrated_full_mueller_polarimetry(cal_angles, lpopt[0], lpopt[1], lpopt[2], lpopt[3], lpopt[4], lpopt[5], cal_left_intensity, LPA_angle=np.pi/2)
    print(MlCal/MlCal.max(), ' Left calibration')
    MrCal = calibrated_full_mueller_polarimetry(cal_angles, rpopt[0], rpopt[1], rpopt[2], rpopt[3], rpopt[4], rpopt[5], cal_right_intensity)
    print(MrCal/MrCal.max(), ' Right calibration')

    # Use the parameters found above from curve fitting to construct the actual Mueller matrix of the sample
    Ml = calibrated_full_mueller_polarimetry(sample_angles, lpopt[0], lpopt[1], lpopt[2], lpopt[3], lpopt[4], lpopt[5], sample_left_intensity, LPA_angle=np.pi/2)
    Ml = Ml/Ml.max()

    Mr = calibrated_full_mueller_polarimetry(sample_angles, rpopt[0], rpopt[1], rpopt[2], rpopt[3], rpopt[4], rpopt[5], sample_right_intensity)
    Mr = Mr/Mr.max()

    np.set_printoptions(suppress=True)

    # Extract retardance from the last entry of the mueller matrix, which should just be cos(phi)
    lretardance = np.arccos(Ml[3,3])/(2*np.pi)
    rretardance = np.arccos(Mr[3,3])/(2*np.pi)
    print(lretardance, ' This is the retardance found from the left spot')
    print(rretardance, ' This is the retardance found from the right spot')

    avg_retardance = (lretardance+rretardance)/2

    return Ml, Mr, avg_retardance

Now lets do 1600 nm, 300 fps, 3.3 tint

In [16]:
# First reduce the calibration data
raw_image = 'DRRP_Cal_1500nm300_3.3_'
dark_image = 'C:\\Users\\EPL User\\Desktop\\DRRP_HWP_Characterization\\Darks\\Dark_300_3.3.fits'
raw_image_folder = r"C:\\Users\\EPL User\\Desktop\\DRRP_HWP_Characterization\\Uncoated_JHK\\Calibrations\\Calibrations_Raw\\Cal_1600_Filter\\"
reduced_image_folder = r"C:\\Users\\EPL User\\Desktop\\DRRP_HWP_Characterization\\Uncoated_JHK\\Calibrations\\Calibrations_Reduced\\Cal_1600_Reduced\\"

dark_subtraction(raw_image, dark_image, raw_image_folder, reduced_image_folder)

In [17]:
reduced_filename = 'Reduced_DRRP_Cal_1500nm300_'
reduced_folder = r"C:\\Users\\EPL User\\Desktop\\DRRP_HWP_Characterization\\Uncoated_JHK\\Calibrations\\Calibrations_Reduced\\Cal_1600_Reduced\\"
lcenter = [258, 252]
rcenter = [258, 329]
maxradius = 9
cutoff = 12000

extracted_data = extract_intensities(reduced_filename, reduced_folder, lcenter, rcenter, maxradius, cutoff)
Cal_Il_1600 = extracted_data[0]
Cal_Ir_1600 = extracted_data[1]

In [18]:
Cal_Il_1600 = np.delete(Cal_Il_1600, [0, 7, 13, 14])
Cal_Ir_1600 = np.delete(Cal_Ir_1600, [0, 7, 13, 14])
Cal_theta1600 = np.delete(theta, [0, 7, 13, 14])

In [19]:
# Now bring in the actual sample data
raw_image = 'DRRP_Uncoated_1600nm_300_'
dark_image = 'C:\\Users\\EPL User\\Desktop\\DRRP_HWP_Characterization\\Darks\\Dark_300_3.3.fits'
raw_image_folder = r"C:\\Users\\EPL User\\Desktop\\DRRP_HWP_Characterization\\Uncoated_JHK\\Uncoated_Raw\\Uncoated_1600_Filter\\"
reduced_image_folder = r"C:\\Users\\EPL User\\Desktop\\DRRP_HWP_Characterization\\Uncoated_JHK\\Uncoated_Reduced\\Uncoated_1600_Reduced\\"

dark_subtraction(raw_image, dark_image, raw_image_folder, reduced_image_folder)

In [20]:
reduced_filename = 'Reduced_DRRP_Uncoated_1600nm_300_'
reduced_folder = r"C:\\Users\\EPL User\\Desktop\\DRRP_HWP_Characterization\\Uncoated_JHK\\Uncoated_Reduced\\Uncoated_1600_Reduced\\"
lcenter = [258, 252]
rcenter = [257, 329]
maxradius = 9
cutoff = 15000

extracted_data = extract_intensities(reduced_filename, reduced_folder, lcenter, rcenter, maxradius, cutoff)
Il_1600 = extracted_data[0]
Ir_1600 = extracted_data[1]

In [21]:
Il_1600 = np.delete(Il_1600, [14, 32, 33, 34])
Ir_1600 = np.delete(Ir_1600, [14, 32, 33, 34])
theta1600 = np.delete(theta, [14, 32, 33, 34])

In [22]:
ultimate_polarimetry(Cal_theta1600, Cal_Il_1600, Cal_Ir_1600, theta1600, Il_1600, Ir_1600)

[ 0.03281     0.0383252   0.05020145 -0.07245235  0.02063271  0.00637496] Left parameters for a1, a2, w1, w2, r1, and r2. 1 for generator, 2 for analyzer
[ 0.04793312  0.0511963   0.06320084 -0.05946429  0.02685385  0.00487121] Right parameters for a1, a2, w1, w2, r1, and r2. 1 for generator, 2 for analyzer
[[ 1.00000000e+00 -6.55649875e-03 -6.93774523e-05  6.36881345e-03]
 [ 7.83346355e-03  9.84451883e-01  2.24047805e-03  5.94041669e-03]
 [-1.47127556e-03  3.98796677e-03  9.82667240e-01 -6.05569324e-04]
 [-1.76521511e-03 -7.37346572e-04  3.91359732e-03  9.97440673e-01]]  Left calibration
[[ 9.98146903e-01 -5.05287539e-03 -7.19405766e-06  5.87711605e-03]
 [-3.29073116e-03  9.83158928e-01  4.21844860e-03  2.87302323e-03]
 [-6.57363603e-03  1.05053522e-02  1.00000000e+00 -1.24754436e-02]
 [ 6.11731788e-03 -6.47098688e-03 -1.36944914e-02  9.88764954e-01]]  Right calibration
0.47950195479571245  This is the retardance found from the left spot
0.46817994285382897  This is the retardance fou

(array([[ 1.        , -0.00509118, -0.00211906, -0.00496846],
        [-0.00016518,  0.98739646,  0.0205981 ,  0.11420561],
        [-0.00861057,  0.02537072, -0.99111006, -0.07208347],
        [ 0.00189128,  0.11450809,  0.07975542, -0.99171764]]),
 array([[ 1.        , -0.0084522 ,  0.00053631,  0.00250051],
        [ 0.00445315,  0.98528583,  0.06206105,  0.11876631],
        [ 0.00545279,  0.07660651, -0.97669765, -0.07288622],
        [-0.00273179,  0.11551411,  0.08307931, -0.98008022]]),
 0.4738409488247707)